<a href="https://colab.research.google.com/github/MichaelKru92/Projekt-ML-Modelierung/blob/main/Schritt_4_Datenexploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# jetzt ist er da# Wo ist der Text

In [ ]:
#Speicherorte:

folderPath = "/Users/fabian/Library/Mobile Documents/com~apple~CloudDocs/BIDS/03_Datenmanagement/Project X /GitHub_V1/Projekt-ML-Modelierung/"      # Fabian
# folderPath = "https://raw.githubusercontent.com/MichaelKru92/Projekt-ML-Modelierung/refs/heads/main/"                                             # GITFolder



# #Benoetigte Libraries werden importiert
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys                                                                                       #os.path bietet Funktionen zur Manipulation von Pfadnamen https://docs.python.org/3/library/os.path.html
import seaborn as sns
import sqlite3


#Aktuelle Python Version wird ausgegeben----------------------------------------
print("Python Version:", sys.version)


#Pfad zum GitHub Repository und Verzeichnisnamen--------------------------------
synthea_url = "https://github.com/MichaelKru92/Projekt-ML-Modelierung"
synthea_dir = "Projekt-ML-Modelierung"
local_Directory = "/Users/fabian/Library/Mobile Documents/com~apple~CloudDocs/BIDS/03_Datenmanagement/Project X /GitHub_V1/Projekt-ML-Modelierung/"

if not os.path.exists(synthea_dir):                                                              #Sofern der Pfad zum Verzeichnis synthea_data noch nicht existiert
    !git clone {synthea_url}                                                                     #Shell command: git kopiere das Repository synthea_url
    print(f"Repository '{synthea_url}' erfolgreich geklont.")
else:
    print(f"Verzeichnis '{synthea_dir}' existiert bereits. Kein erneutes Klonen erforderlich.")


#Inhalte des Verzeichnis inklusiver versteckter Dateien-------------------------
!ls -lah synthea



#Anlegen einer neuen sqlite Datenbank-------------------------------------------
if os.path.exists("synthea.db"):                                                                 #Sofern die Datei synthea.db bereits existiert
    os.remove("synthea.db")                                                                       #Datei wird geloescht
db_name = "synthea.db"                                                                            #DB Dateiname wird vergeben
conn = sqlite3.connect(db_name)                                                                   #Verbindung zur DB Datei wird hergestellt
cur = conn.cursor()                                                                               #Ueber dise Verbindung wird auf die DB ein Cursor gesetzt


OperationalError: no such table: payers

In [ ]:
# Alle Tabellen Laden

CSV_TABLES = ["careplans.csv", "conditions.csv", "devices.csv", "disease.csv", "encounters.csv", "imaging_studies.csv", "immunizations.csv",
              "medications.csv", "observations.csv", "organizations.csv", "patients.csv", "payer_transitions.csv", "payers.csv", "procedures.csv",
              "providers.csv", "supplies.csv"]

counter = 0
while counter < len(CSV_TABLES):
    current_table = pd.read_csv(folderPath + CSV_TABLES[counter])
    print("Tabelle: ", CSV_TABLES[counter], " mit den Attributen: ", current_table.columns)
    counter += 1

print("Alle Tabellennamen und deren Attribute wurden erfolgreich ausgegeben .")



In [ ]:
# CSV Daten in SQL Datenbank laden---------------------------------------------
conn = sqlite3.connect("synthea.db")
cur = conn.cursor()
counter = 0

while counter < len(CSV_TABLES):
    current_table = pd.read_csv(folderPath + CSV_TABLES[counter])
    current_table.to_sql(CSV_TABLES[counter].split(".")[0], conn, if_exists='replace', index = False)
    counter += 1

print("Alle Tabellen wurden erfolgreich in die Datenbank geladen.")


In [ ]:
# Test der Datenbankverbindung-------------------------------------------------

# SQL-Abfrage erstellen
query = "SELECT * from careplans"

# SQL-Abfrage ausführen
cur.execute(query)


# Ergebnisse abrufen
results = cur.fetchall()


# Ergebnisse verarbeiten (zum Beispiel ausdrucken)
for row in results:
    print(row)


In [ ]:
# Funktionierende SQL Abfragen

# SQL-Abfrage erstellen
query = "SELECT Description, count(Description) from disease group by Description order by count(Description) desc"

# SQL-Abfrage ausführen
cur.execute(query)


# Ergebnisse abrufen
results = cur.fetchall()


# Ergebnisse verarbeiten (zum Beispiel ausdrucken)
for row in results:
    print(row)

# Balkendirgram erstellen


In [ ]:
# Allergie Verteilung im Geschlechtervergleich

# SQL-Abfrage erstellen
query1 = "SELECT Description, count(Description) from disease, patients where disease.patient = patients.id and patients.gender = 'M' group by Description order by count(Description) desc"
query2 = "SELECT Description, count(Description) from disease, patients where disease.patient = patients.id and patients.gender = 'F' group by Description order by count(Description) desc"

# SQL-Abfrage ausführen
cur.execute(query1)
result1 = cur.fetchall()

# Ergebnisse abrufen
cur.execute(query2)
result2 = cur.fetchall()

# Ergebnisse verarbeiten (zum Beispiel ausdrucken)
#for row in result1:
#    print(row)

#for row in result2:
#    print(row)

# Balkendirgram erstellen
df1m = pd.DataFrame(result1, columns=['Disease', 'Count'])
df2f = pd.DataFrame(result2, columns=['Disease', 'Count'])
df1m.plot(kind='bar', x='Disease', y='Count', color='blue', title='Männlich Allergie Verteilung')
df2f.plot(kind='bar', x='Disease', y='Count', color='red', title='Weiblich Allergie Verteilung')

# gruppiertes Balkendiagramm erstellen

In [ ]:
# Allergie Verteilung im Geschlechtervergleich

# SQL-Abfrage erstellen
query1 = "SELECT careplans.Description, count(careplans.Description) from careplans group by careplans.Description order by count(careplans.Description) desc"

# SQL-Abfrage ausführen
cur.execute(query1)
result1 = cur.fetchall()


# Ergebnisse verarbeiten (zum Beispiel ausdrucken)
#for row in result1:
#    print(row)


# Balkendirgram erstellen
df1m = pd.DataFrame(result1, columns=['Disease', 'Count'])

df1m.plot(kind='bar', x='Disease', y='Count', color='blue', title='Verteilung der Versorgungen')

In [ ]:
# Allergie Verteilung im Geschlechtervergleich

# SQL-Abfrage erstellen
query1 = "SELECT payers.name, count(encounters.id), sum(encounters.TOTAL_CLAIM_COST) from payers, encounters where payers.id = encounters.payer group by payers.name order by sum(encounters.TOTAL_CLAIM_COST) desc"

# SQL-Abfrage ausführen
cur.execute(query1)
result1 = cur.fetchall()

print(result1)


# Ergebnisse verarbeiten (zum Beispiel ausdrucken)
for row in result1:
    print(row)


# Balkendirgram erstellen
df1m = pd.DataFrame(result1, columns=['Insuarence', 'Count', 'Total Cost'])

df1m.plot(kind='bar', x='Insuarence', y='Count', color='blue', title='Verteilung der Versorgungen')
